参考  
aidiaryさんのブログ : http://aidiary.hatenablog.com/entry/20180129/1517233796  
pytorch tutorial : http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

In [1]:
import numpy as np
import torch
from torch.autograd import Variable

Variable classにあるbackwardメソッドを呼ぶことで導関数を求めることができる．  
Variableでrequires_grad=Trueにすると導関数を計算することができる．  
requires_grad=FalseだとgradでNoneを返す．

In [2]:
x=Variable(torch.ones(2,2), requires_grad=True)
w=torch.ones(2,1)
w.fill_(3.0)
w=Variable(w, requires_grad=True)
print(x, w)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]
 Variable containing:
 3
 3
[torch.FloatTensor of size 2x1]



In [3]:
y=x+2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [4]:
print(y.grad_fn)

In [5]:
z=y*y*3
out=z.mean()

In [6]:
out.backward()

In [7]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



In [8]:
import torch.nn as nn
import torch.nn.functional as F

In [9]:
"""
import numpy as np
x = Variable(torch.Tensor([2]), requires_grad=True)
x = np.exp(x)
print(type(x))
y = torch.from_numpy(x)
y.backward()
print(x.grad)
"""

'\nimport numpy as np\nx = Variable(torch.Tensor([2]), requires_grad=True)\nx = np.exp(x)\nprint(type(x))\ny = torch.from_numpy(x)\ny.backward()\nprint(x.grad)\n'

numpyに変換する関数があるけど上記のようにexpとかsinとかを変換するのには使えない  
だからtorchの中にある関数でやる必要がある  
上記のやつは下記のようにする

In [10]:
x = Variable(torch.Tensor([2]), requires_grad=True)
y = torch.exp(x)
y.backward()
print(x.grad)

Variable containing:
 7.3891
[torch.FloatTensor of size 1]



sin,cos,squrなども同様  
- sinの微分
$$
y = \sin(x) \\
\frac{dy}{dx} = \cos(x)
$$

- cosの微分
$$
y = \cos(x)\\
\frac{dy}{dx}=-\sin(x)
$$

- sqrtの微分
$$
y=\sqrt{x}\\
\frac{dy}{dx}=\frac{1}{2\sqrt{x}}
$$

- expの微分
$$
y=\mathrm{e}^{x}\\
\frac{dy}{dx} = \mathrm{e}^{x}
$$

In [11]:
#sin
x = Variable(torch.Tensor([2]), requires_grad=True)
y = torch.sin(x)
z = torch.cos(x)
y.backward()
print(x.grad, z)

#cos
x = Variable(torch.Tensor([2]), requires_grad=True)
y = torch.cos(x)
z = torch.sin(-x)
y.backward()
print(x.grad, z)

#sqrt
x = Variable(torch.Tensor([2]), requires_grad=True)
y = torch.sqrt(x)
z = 1/(2*torch.sqrt(x))
y.backward()
print(x.grad, z)

#exp
x = Variable(torch.Tensor([2]), requires_grad=True)
y = torch.exp(x)
y.backward()
print(x.grad, y)

Variable containing:
-0.4161
[torch.FloatTensor of size 1]
 Variable containing:
-0.4161
[torch.FloatTensor of size 1]

Variable containing:
-0.9093
[torch.FloatTensor of size 1]
 Variable containing:
-0.9093
[torch.FloatTensor of size 1]

Variable containing:
 0.3536
[torch.FloatTensor of size 1]
 Variable containing:
 0.3536
[torch.FloatTensor of size 1]

Variable containing:
 7.3891
[torch.FloatTensor of size 1]
 Variable containing:
 7.3891
[torch.FloatTensor of size 1]



loss(二乗誤差)を微分する

In [14]:
import torch.nn as nn
import torch.nn.functional as F

# 3unitから2unitのlinear層(全結合層)
linear = nn.Linear(3, 2)

print('w:', linear.weight)
print('b:', linear.bias)

# 誤差関数(損失関数)：二乗誤差
criterion = nn.MSELoss()

# 最適化アルゴリズム：勾配降下法(SGD)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# バッチ5，入力3，出力2
input = Variable(torch.randn(5, 3)) # 入力
target = Variable(torch.randn(5, 2)) # 教師
output = linear(input) # 入力からの出力
loss = criterion(output, target) # 誤差関数の値
print('loss:', loss)

# backpropagation
loss.backward()

print('dL/dw:', linear.weight.grad)
print('dL/db:', linear.bias.grad)

# 学習可能なモデルのパラメータはlinear.parameters()のよって返される
print('---learnable params---')
params = list(linear.parameters())
print('params:', len(params))
print('first param size', params[0].size())

print('\n---by hand---')
learning_rate = 0.01
print(linear.weight.sub(linear.weight.grad * learning_rate))
print(linear.bias.sub(linear.bias.grad * learning_rate))

optimizer.step()    # Does the update
print('---optimizer step---')
print('w:', linear.weight)
print('b:', linear.bias)

w: Parameter containing:
-0.4080  0.1045  0.0191
 0.4515  0.4651 -0.4163
[torch.FloatTensor of size 2x3]

b: Parameter containing:
 0.2342
 0.2078
[torch.FloatTensor of size 2]

loss: Variable containing:
 0.8999
[torch.FloatTensor of size 1]

dL/dw: Variable containing:
-0.3601 -0.0032  0.4757
 0.0399  0.4923 -0.6611
[torch.FloatTensor of size 2x3]

dL/db: Variable containing:
 0.3013
-0.7033
[torch.FloatTensor of size 2]

---learnable params---
params: 2
first param size torch.Size([2, 3])
---by hand---
Variable containing:
-0.4044  0.1045  0.0144
 0.4511  0.4602 -0.4097
[torch.FloatTensor of size 2x3]

Variable containing:
 0.2311
 0.2148
[torch.FloatTensor of size 2]

---optimizer step---
w: Parameter containing:
-0.4044  0.1045  0.0144
 0.4511  0.4602 -0.4097
[torch.FloatTensor of size 2x3]

b: Parameter containing:
 0.2311
 0.2148
[torch.FloatTensor of size 2]

